In [18]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import synapseclient
from synapseclient import Project, Folder, File, Link
import time

In [2]:
#will login to synapse

syn = synapseclient.login()
syn.login()

Welcome, nmoyen@stanford.edu!

Welcome, nmoyen@stanford.edu!



In [3]:
syn_id = 'syn4216032' #id for validation data for gps files- can change to any ID

In [4]:
#function to pull data from tables on synapse; the "table_results" create a list of json files (that you will then need
#to read in), while the df_table is a dataframe where you can pull healthCode data or any other data needed to add into
#individual json files.

def get_table(syn_id):
    table_id = syn_id
    table_results = syn.tableQuery("SELECT * FROM %s" % table_id, resultsAs="rowset")
    df_table = table_results.asDataFrame()
    return table_results, df_table

In [5]:
#running function
table_results, df_table = get_table(syn_id)

In [6]:
#this reads the json files from a specified column. In this case, the gps files from the validation study (+a few others
#that we'll have to filter out later by created on datetime.
files_gps = []
for x in range(771,797): #can change this range to pull any gps files; might be limited by API calls depending on #
    files_gps.append(syn.downloadTableFile(table_results, column='UnknownFile_1.json.items', rowId=x, versionNumber=1, ifcollision=u'keep.both'))

Requests are too frequent for API call: /entity/#/column. Allowed 6 requests every 60 seconds.
Requests are too frequent for API call: /entity/#/column. Allowed 6 requests every 60 seconds.
Requests are too frequent for API call: /entity/#/column. Allowed 6 requests every 60 seconds.


In [7]:
#pulling in data from synapse table (df_table) needed to merge with validation data google sheet
times_zone = pd.DataFrame(df_table['createdOnTimeZone'][771:797])
times_created = pd.DataFrame(df_table['createdOn'][771:797])
appV = pd.DataFrame(df_table['appVersion'][771:797])

In [28]:
files_gps[-1]

u'/Users/nicolemoyen/.synapseCache/324/18425324/UnknownFile_1.json.items-2d905291-c068-4758-89c6-7f89ce97dc3b.json'

In [8]:
#this reads in the json files and creates a list of dataframes of the files you imported
gps_valdf = []

for i in range(len(files_gps)):
    gps_valdf.append(pd.read_json(files_gps[i]))
    gps_valdf[i]['timezone'] = times_zone['createdOnTimeZone'][i]
    gps_valdf[i]['createdOn'] = times_created['createdOn'][i]


In [12]:
'''this function first creates a new "start time", which is the original timestamp shifted down one row. This allows
for subtracting of start_time and timestamp (i.e. endtime) to get the difference in timestamps for each sample (in sec)
so that can compute "feasible" time to walk xx meters. The next part of the function says if that sample is 1 second, 
then only keep displacement samples <4.0 meters (this is probably the physiologically possible cutoff). If the sample
lasts more than 1 second, then it will multiply that number of seconds * 4(for 4 meters/second) to get a new threshold
that someone could feasibly walk in that amount of time. It only keeps displacement values below this threshold. 
Last, it creates a dataframe with the "new_distance: what's been calculated from this "fix", and the "gps_reported" 
which is the sum of the displacement values with no data cleaning.'''

'''This argument takes "df" which is a list of gps files (already converted from json to dataframe), it will then
iterrate through each of the rows in each file.'''

def clean_gps(df):
    
    dist_df = []
    total_time = []

    for i in range(len(df)):
        df[i]['start_time'] = df[i]['timestamp'].shift(1)
        df[i]['change']= ((df[i]['timestamp'] - df[i]['start_time']))
        total_time.append(df[i].timestamp[-1:] - df[i].timestamp[0])

        dist = []
        new_thres = []

        for index,row in df[i].iterrows():      
            seconds = (row.change).seconds
            if seconds == 'nan':
                continue
            elif (row.change).seconds <2:
                if row.displacement <4.:
                    dist.append(row.displacement)
            else:
                new_thres.append(seconds*4)
                if row.displacement <new_thres:
                    dist.append(row.displacement)

        #create df with new distance, reported displacement, healthcode, created on, and timezone
        df_new = {'new_distance': sum(dist),
                 'gps_reported': sum(df[i]['displacement']), 
                 'total time of test': total_time[i],
                 'createdOn': df[i].createdOn[1],
                 'timezone': df[i].timezone[1]}
        dist_df.append(df_new)
    return df, dist_df

In [13]:
#calling above function
df,dist_df = clean_gps(gps_valdf)

In [16]:
#convert to df so can change more times and also download as csv if wanted
overall_gps = pd.DataFrame.from_dict(dist_df,orient='columns')

In [21]:
#adding a column where convert created On timestamp to PST so we can merge with ground truth measurements.
overall_gps['PST'] = (overall_gps['createdOn'] +  timedelta(hours=-7))

#converting timestamp to match that in truth distances
PST = []

for index,row in overall_gps.iterrows():
    PST.append(datetime.strftime(row.PST,"%-m/%d/%y %H:%M")) #convert it to matching time on google sheet
    
overall_gps['timetouse'] = PST

In [26]:
overall_gps # now we can filter out the 17 subjects from validation data only using the "timetouse" info

,createdOn,gps_reported,new_distance,timezone,total time of test,PST,timetouse
0,2017-09-20 16:22:56,422.403108,194.083990,-0700,"70 00:01:37 Name: timestamp, dtype: timedelt...",2017-09-20 09:22:56,9/20/17 09:22
1,2017-09-20 16:40:22,769.766980,318.691363,-0700,"86 00:02:05 Name: timestamp, dtype: timedelt...",2017-09-20 09:40:22,9/20/17 09:40
2,2017-09-20 16:40:23,1526.263395,1081.079293,-0700,"363 00:10:42 Name: timestamp, dtype: timedel...",2017-09-20 09:40:23,9/20/17 09:40
3,2017-09-20 16:51:55,187.874386,145.978475,-0700,"73 00:01:46 Name: timestamp, dtype: timedelt...",2017-09-20 09:51:55,9/20/17 09:51
4,2017-09-20 17:06:07,1327.770803,833.889573,-0700,"364 00:10:26 Name: timestamp, dtype: timedel...",2017-09-20 10:06:07,9/20/17 10:06
5,2017-09-20 17:06:58,648.391315,244.874129,-0700,"75 00:04:28 Name: timestamp, dtype: timedelt...",2017-09-20 10:06:58,9/20/17 10:06
6,2017-09-20 17:19:40,1324.632540,851.820202,-0700,"364 00:08:05 Name: timestamp, dtype: timedel...",2017-09-20 10:19:40,9/20/17 10:19
7,2017-09-20 17:20:10,1141.546083,763.298561,-0700,"363 00:07:51 Name: timestamp, dtype: timedel...",2017-09-20 10:20:10,9/20/17 10:20
8,2017-09-20 17:43:43,429.715948,168.131014,-0700,"87 00:03:46 Name: timestamp, dtype: timedelt...",2017-09-20 10:43:43,9/20/17 10:43
9,2017-09-20 17:57:15,1402.388070,941.779923,-0700,"363 00:06:48 Name: timestamp, dtype: timedel...",2017-09-20 10:57:15,9/20/17 10:57


In [27]:
overall_gps.to_csv('/Users/nicolemoyen/Dropbox/Stanford/AshleyLab/validation_distances.csv')